<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/images/skip_connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following logic will turn 0 at a certain iter due to vanishing gadients problem.

In [19]:
import numpy as np
from scipy.signal import convolve2d

image = np.ones((8,8), dtype=np.float32)
kernel = np.array([[0.05, 0.10, 0.05],
                   [0.10, 0.20, 0.10],
                   [0.05, 0.10, 0.05]], dtype=np.float32)

for i in range(1000):
  image = convolve2d(image, kernel, mode='same', boundary='fill', fillvalue=0)

  if (image > 0).sum() == 0:
    print ('All value are zero at iter = ', i)
    break

All value are zero at iter =  362


To solve this issue we could use skip connection and add in numerical stability ( softmax ) using resnet method.

In [23]:
import numpy as np
from scipy.signal import convolve2d

def softmax2d(x):
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x)


image = np.ones((8,8), dtype=np.float32)
kernel = np.array([[0.05, 0.10, 0.05],
                   [0.10, 0.20, 0.10],
                   [0.05, 0.10, 0.05]], dtype=np.float32)

for i in range(1000):
  residual = image.copy()
  image = convolve2d(image, kernel, mode='same', boundary='fill', fillvalue=0)
  image = softmax2d(residual + image)

  if (image > 0).sum() == 0:
    print ('All value are zero at iter = ', i)
    break

print ('Iterations complete. \noutput = ')
image

Iterations complete. 
output = 


array([[0.01556201, 0.01559918, 0.01559928, 0.01559928, 0.01559928,
        0.01559928, 0.01559918, 0.01556201],
       [0.01559918, 0.01564896, 0.01564911, 0.01564912, 0.01564912,
        0.01564911, 0.01564896, 0.01559918],
       [0.01559928, 0.01564911, 0.01564927, 0.01564928, 0.01564928,
        0.01564927, 0.01564911, 0.01559928],
       [0.01559928, 0.01564912, 0.01564928, 0.01564928, 0.01564928,
        0.01564928, 0.01564912, 0.01559928],
       [0.01559928, 0.01564912, 0.01564928, 0.01564928, 0.01564928,
        0.01564928, 0.01564911, 0.01559928],
       [0.01559928, 0.01564911, 0.01564927, 0.01564928, 0.01564928,
        0.01564927, 0.01564911, 0.01559928],
       [0.01559918, 0.01564896, 0.01564911, 0.01564911, 0.01564911,
        0.01564911, 0.01564896, 0.01559918],
       [0.01556201, 0.01559918, 0.01559928, 0.01559928, 0.01559928,
        0.01559928, 0.01559918, 0.01556201]], dtype=float32)

A densenet arch will allow the concatenation of all outputs so far in the stack with adding numerical stability for example, softmax. This is also a variant of skip connections.

In [33]:
import numpy as np
from scipy.signal import convolve2d

image = np.ones((8, 8), dtype=np.float32)
kernel = np.array([[0.05, 0.10, 0.05],
                   [0.10, 0.20, 0.10],
                   [0.05, 0.10, 0.05]], dtype=np.float32)

features = [image]

for i in range(1000):
    concatenated = np.stack(features, axis=0).sum(axis=0)
    out = convolve2d(concatenated, kernel, mode='same', boundary='fill', fillvalue=0)

    out = softmax2d(out)
    features.append(out)

    # Optional: stop if things explode
    if np.isinf(out).any() or np.isnan(out).any():
        print(f"Numerical instability at iteration {i}")
        break

print ('Iterations complete. \nOutput =', out)

Iterations complete. 
Output = [[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.4251205e-42 9.2476806e-30 1.4026722e-37
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 4.6814219e-30 1.0000000e+00 1.9949366e-16
  1.4012985e-45 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 3.5945632e-38 2.5879985e-17 2.9516086e-25
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00]]
